In [416]:
import pandas as pd
import numpy as np


In [417]:
xlsx = pd.ExcelFile("DataSet.xlsx")
df = pd.read_excel(xlsx)



In [418]:
#Monthly return sp500
df['sp500ret']= df['SP500Index'].pct_change()

In [419]:
#Monthly return of bond index
df['bondret'] = df['BarclaysUSAggTreasury Index'].pct_change()


In [421]:
#Benchmark Portfolio
df['benchmarkret'] = .6*df['sp500ret'] + .4*df['bondret']


In [422]:
#adding 1 to returns for caluclation purposes
df['1+sp500ret']= 1+ df['sp500ret']
df['1+bondret']=1+ df['bondret']
df['1+benchmarkret'] = 1 + df['benchmarkret']


In [424]:
#Total Compound Cumulative Returns, 
#sp500CumRet = (df["SP500Index"][len(df.index)-1] / df['SP500Index'][0]) -1
#bondCumRet = (df['BarclaysUSAggTreasury Index'][len(df.index)-1] / df['BarclaysUSAggTreasury Index'][0])
df['sp500CumRet']= df['1+sp500ret'].cumprod()-1
df['bondCumRet'] = df['1+bondret'].cumprod()-1
df['benchmarkCumRet'] = df['1+benchmarkret'].cumprod()-1



In [425]:
#Annualized Returns, 
sp500AnnRet = (df["sp500CumRet"][len(df.index)-1])**(1/27.)-1
bondAnnRet = (df["bondCumRet"][len(df.index)-1])**(1/27.)-1
benchmarkAnnRet = (df["benchmarkCumRet"][len(df.index)-1])**(1/27.)-1
print sp500AnnRet, bondAnnRet, benchmarkAnnRet

0.0931989940909 0.0492857949103 0.0797343968425


In [426]:
#Annual Standard Deviation
sp500SD = np.std(df['sp500ret'])*np.sqrt(12)
bondSD = np.std(df['bondret'])*np.sqrt(12)
benchmarkSD = np.std(df['benchmarkret'])*np.sqrt(12)
print sp500SD, bondSD, benchmarkSD

0.14367923596 0.044111172661 0.086196221143


In [427]:
#annual return of risk free for sharpe ratio calculation
cumRetRF = (df['3m Tbill Index'][len(df.index)-1] /df['3m Tbill Index'][0]) - 1
rfAnnRet = cumRetRF**(1/27.) - 1
rfAnnRet

0.007419355012572959

In [428]:
#sharpe ratio
sharpeRatio = (benchmarkAnnRet - rfAnnRet )/ benchmarkSD
sharpeRatio

0.83895837742055102

In [429]:
#making new dataframe for questions 3+
df2 = df.iloc[:,0:10]

In [431]:
#Finding returns of the factor
for i in range(0,4):
    df2['factor{}ret'.format(i+1)] = df2['Factor{}'.format(i+1)].pct_change()


In [432]:
#Lagging returns of the factors for prediction purposes, shift factor returns one month down.
for i in range(0,4):
    df2['laggedf{}ret'.format(i+1)] = df2['factor{}ret'.format(i+1)].shift(1)

In [433]:
#Calculating regression based off of last 3 years.(factor beta)
for i in range(0,4):
    df2['beta{}'.format(i+1)] = df2['sp500ret'].rolling(window=36).cov(other=df2['laggedf{}ret'.format(i+1)])/ df2['laggedf{}ret'.format(i+1)].rolling(window=36,center=False).var()


In [434]:
#average of sp500 returns from past 3 years
df2['sp500RollingMean'] = df2['sp500ret'].rolling(window=36).mean()

In [435]:
#average of lagged return from past 3 years
for i in range(0,4):
    df2['laggedf{}RollingMean'.format(i+1)] = df2['laggedf{}ret'.format(i+1)].rolling(window=36).mean()


In [436]:
# finding intercept of linear regression
for i in range(0,4):
    df2['f{}intercept'.format(i+1)] = df2['sp500RollingMean'] - df2['beta{}'.format(i+1)]*df2['laggedf{}RollingMean'.format(i+1)]

In [437]:
#Predicted Return( y=mx+b) 
for i in range(0,4):
    df2['f{}PredictRet'.format(i+1)] = df2['f{}intercept'.format(i+1)] + df2['beta{}'.format(i+1)]*df2['laggedf{}ret'.format(i+1)]

In [439]:
#error squared, difference between predicted return and actual return
for i in range(0,4):
    df2['f{}errorSquare'.format(i+1)] = (df2['f{}PredictRet'.format(i+1)]- df2['sp500ret'])**2

In [440]:
#Mean Squared Error, lowest MSE is factor chosen.
# using n-k-1 for denominator of MSE

df2['n-k-1'] = df2.index - 3

print df2['f1errorSquare'].mean(), df2['f2errorSquare'].mean(), df2['f3errorSquare'].mean(), df2['f4errorSquare'].mean()

#Factor 3 is best.

0.00159828956328 0.0016335861836 0.00142505170127 0.00154764189865


In [441]:
#Rebalance formula

df2['equityWeight']=.6


for i in range(37,len(df.index)):
    rebalanceFormula = 1.5 - (1/(1+np.exp(df2['f3PredictRet'][i])))

    if df2['f3PredictRet'][i] >= 0:
        if df2['equityWeight'][i-1]* rebalanceFormula >= .7:
            df2['equityWeight'][i] = .7
        else:
            df2['equityWeight'][i] = df2['equityWeight'][i-1]* rebalanceFormula
    else:
        if df2['equityWeight'][i-1]* rebalanceFormula <= .5:
            df2['equityWeight'][i] = .5
        else:
            df2['equityWeight'][i] = df2['equityWeight'][i-1]*rebalanceFormula


C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [442]:
#calculating bondweight of portfolio
df2['bondWeight'] = 1 - df2['equityWeight']


In [443]:
#Setting first couple of rows of equity/bond weights as 0; don't have appropriate betas during this time period
df2['equityWeight'][:37] = None
df2['bondWeight'][:37] = None


C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [444]:
# portfolio return
df2['portfolioRet'] = df2['bondWeight']*df2['bondret'] + df2['equityWeight']*df2['sp500ret']

In [445]:
#alpha
df2['alpha'] = df2['portfolioRet'] - df['benchmarkret']


In [446]:
#cumulative alpha
cumAlpha = (1 + df2['alpha']).prod() - 1
cumAlpha

0.10937102120070974

In [447]:
#annualized alpha
n = 287./12 #of periods 27 years - 3 years for beta and 1 day for lag
annualAlpha = (1+ cumAlpha)**(1/n) - 1
annualAlpha


0.0043492161718881039

In [448]:
#tracking error
#df2['alpha'].std()#, also tracking error 
numMonths = 286
trackingError = ((df2['alpha']**2).sum()/numMonths)**(1/2.)
trackingError
            

0.0041926336571421925

In [449]:
#Information Ratio
infoRatio = annualAlpha/ trackingError
infoRatio

1.0373470537973599

In [450]:
#random initial portfolio value
df2['portfolioValue'] = 1000000

In [451]:
for i in range(37, len(df2.index)):
    df2['portfolioValue'][i] = df2['portfolioValue'][i-1]* (1+df2['portfolioRet'][i])

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [452]:
df2

,Date,SP500Index,BarclaysUSAggTreasury Index,3m Tbill Index,Factor1,Factor2,Factor3,Factor4,sp500ret,bondret,...,f1errorSquare,f2errorSquare,f3errorSquare,f4errorSquare,n-k-1,equityWeight,bondWeight,portfolioRet,alpha,portfolioValue
0,1990-01-31,353.94,460.650115,311.712356,25.36,127.500,111.53,47.2,NaN,NaN,...,NaN,NaN,NaN,NaN,-3,NaN,NaN,NaN,NaN,1000000
1,1990-02-28,358.50,461.479285,313.593438,21.99,128.000,112.95,49.1,0.012884,0.001800,...,NaN,NaN,NaN,NaN,-2,NaN,NaN,NaN,NaN,1000000
2,1990-03-30,368.00,461.340841,315.663041,19.73,128.600,114.35,49.9,0.026499,-0.000300,...,NaN,NaN,NaN,NaN,-1,NaN,NaN,NaN,NaN,1000000
3,1990-04-30,358.81,457.096505,317.781562,19.52,128.900,123.82,50.0,-0.024973,-0.009200,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,1000000
4,1990-05-31,393.80,469.940917,319.906072,17.37,129.100,113.91,49.5,0.097517,0.028100,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,1000000
5,1990-06-29,391.14,477.365984,321.902140,15.50,129.900,122.38,49.2,-0.006755,0.015800,...,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,1000000
6,1990-07-31,389.89,483.428532,324.093456,21.11,130.500,125.26,46.6,-0.003196,0.012700,...,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,1000000
7,1990-08-31,354.65,476.467161,326.176669,29.90,131.600,118.57,46.1,-0.090384,-0.014400,...,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,1000000
8,1990-09-28,337.39,481.041245,328.051685,29.11,132.500,117.36,44.5,-0.048668,0.009600,...,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,1000000
9,1990-10-31,335.95,488.978426,330.210259,30.04,133.400,121.64,43.2,-0.004268,0.016500,...,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,1000000


In [453]:
writer = pd.ExcelWriter('Wellington.xlsx')
df.to_excel(writer,'Sheet1')
df2.to_excel(writer,'Sheet2')
writer.save()